In [6]:
import numpy as np
import csv
import os
from datetime import datetime, timezone
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [7]:
date = '2023_07_04'
rover_type = 'Trimble_R10/'
path = os.path.expanduser('~') + '/ros2_ws/src/ICRA_2024/GNSS_Data/' + rover_type + date + '/'

In [8]:
# Inverse North for East and compute timestamp

file_pairs = [
    (path + 'rover_1_raw.csv', path + 'rover_1.csv'),
    (path + 'rover_2_raw.csv', path + 'rover_2.csv'),
    (path + 'rover_3_raw.csv', path + 'rover_3.csv')
]

# Boucle for pour parcourir les fichiers
for input_file, output_file in file_pairs:

    with open(input_file, 'r') as file:
        reader = csv.reader(file)
        lines = list(reader)

    header = lines.pop(0)


    for line in lines:
        temps_str = line[7]
        date_str = line[8]
        
        format_str = "%H:%M:%S,%Y-%m-%d"
        temps_date_str = f"{temps_str},{date_str}"
        
        temps_objet = datetime.strptime(temps_date_str, format_str)
        
        temps_objet_utc = temps_objet.replace(tzinfo=timezone.utc)
        
        timestamp_unix = int(temps_objet_utc.timestamp())
        
        line[7] = timestamp_unix
        line[8] = date_str
        del line[9] # empty column

    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(lines)

    print("Opérations terminées. Le fichier mis à jour est enregistré sous", output_file)

Opérations terminées. Le fichier mis à jour est enregistré sous /home/effie/ros2_ws/src/ICRA_2024/GNSS_Data/Trimble_R10/2023_07_04/rover_1.csv
Opérations terminées. Le fichier mis à jour est enregistré sous /home/effie/ros2_ws/src/ICRA_2024/GNSS_Data/Trimble_R10/2023_07_04/rover_2.csv
Opérations terminées. Le fichier mis à jour est enregistré sous /home/effie/ros2_ws/src/ICRA_2024/GNSS_Data/Trimble_R10/2023_07_04/rover_3.csv


In [20]:
rovers = {
    'rover_1': {'Id': [], 'East': [], 'North': [], 'Up': [], 'Sigma_H': [],'Sigma_V': [], 'PDOP': [], 'nSats': [], 'Timestamp': []},
    'rover_2': {'Id': [], 'East': [], 'North': [], 'Up': [], 'Sigma_H': [],'Sigma_V': [], 'PDOP': [], 'nSats': [], 'Timestamp': []},
    'rover_3': {'Id': [], 'East': [], 'North': [], 'Up': [], 'Sigma_H': [],'Sigma_V': [], 'PDOP': [], 'nSats': [], 'Timestamp': []}

}
current_rover = None
count = 0
all_rover_data = {}

for rover, data in rovers.items():
    rover_number = rover.split("_")[1]
    filename = path + rover + '.csv'
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(reader)
        next(reader)

        if current_rover != rover:
            current_rover = rover
            count = 0
        for row in reader:
            id_number = int(row[0].split('nor' + str(rover_number) + '-')[1])
            Id = float(id_number)
            East = float(row[1])
            North = float(row[2])
            Up = float(row[3])
            Timestamp = float(row[7])
            Date = str(row[8])
            array = np.array([Id, North, East, Up, Timestamp])
            data['Id'].append(Id)
            data['North'].append(North)
            data['East'].append(East)
            data['Up'].append(Up)
            data['Timestamp'].append(Timestamp)
            count += 1

    rover_df = pd.DataFrame(data)
    all_rover_data[rover] = rover_df
    # print(all_rover_data['rover_1'])


1


In [ ]:
rover_name = zip('rover_1', 'rover_2', 'rover3')
for rover in rover_name : 
    rover_df = all_rover_data[rover_name]
    east_coordinates = np.array(rover_df['East'])
    north_coordinates = np.array(rover_df['North'])
    up_coordinates = np.array(rover_df['Up'])
    horizontal_error = np.array(rover_df['Sigma_H'])
    vertical_error = np.array(rover_df['Sigma_V'])
    PDOP = np.array(rover_df['PDOP'])
    satellites_number = np.array(rover_df['nSats'])
    timestamps = np.array(rover_df['Timestamp'])